# Import libraries and define variables

In [4]:
!mkdir calculations

In [1]:
import numpy as np
import pandas as pd
import pickle
import gc

In [2]:
dataCols = ['i_tp', 'i_fp', 'i_tn', 'i_fn', 'j_tp', 'j_fp', 'j_tn', 'j_fn']
calculationsDir = "calculations/"
datasetName = "Set(08,56).bin"

# Get calculations
As the dataset is quite large (4.2 Gb) we will write calculations to separate files in 2 stages.

In [3]:
def getGroupRatios(df):
    return (df.j_tp + df.j_fp + df.j_tn + df.j_fn) / (df.i_tp + df.i_fp + df.i_tn + df.i_fn + df.j_tp + df.j_fp + df.j_tn + df.j_fn)

def getImbalanceRatios(df):
    return (df.i_tp + df.i_fn + df.j_tp + df.j_fn) / (df.i_tp + df.i_fp + df.i_tn + df.i_fn + df.j_tp + df.j_fp + df.j_tn + df.j_fn)

def getTPR_i(df):
    return df.i_tp / (df.i_tp + df.i_fn)

def getTPR_j(df):
    return df.j_tp / (df.j_tp + df.j_fn)

def getFPR_i(df):
    return df.i_fp / (df.i_fp + df.i_tn)

def getFPR_j(df):
    return df.j_fp / (df.j_fp + df.j_tn)

def getPositivePredictiveValue_i(df):
    return df.i_tp / (df.i_tp + df.i_fp)

def getPositivePredictiveValue_j(df):
    return df.j_tp / (df.j_tp + df.j_fp)

def getNegativePredictiveValue_i(df):
    return df.i_tn / (df.i_tn + df.i_fn)

def getNegativePredictiveValue_j(df):
    return df.j_tn / (df.j_tn + df.j_fn)

#each group has the same probability of being classified with positive outcome
def get_statistical_parity(df):
    return ((df.j_tp + df.j_fp)/(df.j_tp + df.j_fp + df.j_tn + df.j_fn)) - ((df.i_tp + df.i_fp)/(df.i_tp + df.i_fp + df.i_tn + df.i_fn))

#similiar to statistical parity, but using ratio
def get_disparate_impact(df):
    return ((df.j_tp + df.j_fp)/(df.j_tp + df.j_fp + df.j_tn + df.j_fn)) / ((df.i_tp + df.i_fp)/(df.i_tp + df.i_fp + df.i_tn + df.i_fn))

#accuracy equality ratio
def get_acc_equality_ratio(df):
    return ((df.j_tp + df.j_tn)/(df.j_tp + df.j_fp + df.j_tn + df.j_fn)) / ((df.i_tp + df.i_tn)/(df.i_tp + df.i_fp + df.i_tn + df.i_fn))

#accuracy equality difference
def get_acc_equality_diff(df):
    return ((df.j_tp + df.j_tn)/(df.j_tp + df.j_fp + df.j_tn + df.j_fn)) - ((df.i_tp + df.i_tn)/(df.i_tp + df.i_fp + df.i_tn + df.i_fn))

## Write calculations of the 1st half of the dataset

In [8]:
# Get half of the data
with open(datasetName, "rb") as f:
    df = pd.DataFrame(pickle.load(f), columns = dataCols)

halfIdx = int(df.shape[0] / 2)
df = df.iloc[:halfIdx]
df.head()

,i_tp,i_fp,i_tn,i_fn,j_tp,j_fp,j_tn,j_fn
0,56,0,0,0,0,0,0,0
1,55,1,0,0,0,0,0,0
2,55,0,1,0,0,0,0,0
3,55,0,0,1,0,0,0,0
4,55,0,0,0,1,0,0,0


In [9]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276635335 entries, 0 to 276635334
Data columns (total 8 columns):
 #   Column  Dtype
---  ------  -----
 0   i_tp    int8 
 1   i_fp    int8 
 2   i_tn    int8 
 3   i_fn    int8 
 4   j_tp    int8 
 5   j_fp    int8 
 6   j_tn    int8 
 7   j_fn    int8 
dtypes: int8(8)
memory usage: 2.1 GB


In [10]:
# Calculate half of GRs
with open(calculationsDir + "gr.bin", "wb+") as f:
    getGroupRatios(df).to_numpy().tofile(f)

# Calculate half of IRs
with open(calculationsDir + "ir.bin", "wb+") as f:
    getImbalanceRatios(df).to_numpy().tofile(f)

with open(calculationsDir + "i_tpr.bin", "wb+") as f:
    getTPR_i(df).to_numpy().tofile(f)

with open(calculationsDir + "j_tpr.bin", "wb+") as f:
    getTPR_j(df).to_numpy().tofile(f)

with open(calculationsDir + "i_fpr.bin", "wb+") as f:
    getFPR_i(df).to_numpy().tofile(f)

with open(calculationsDir + "j_fpr.bin", "wb+") as f:
    getFPR_j(df).to_numpy().tofile(f)

with open(calculationsDir + "i_ppv.bin", "wb+") as f:
    getPositivePredictiveValue_i(df).to_numpy().tofile(f)

with open(calculationsDir + "j_ppv.bin", "wb+") as f:
    getPositivePredictiveValue_j(df).to_numpy().tofile(f)

with open(calculationsDir + "i_npv.bin", "wb+") as f:
    getNegativePredictiveValue_i(df).to_numpy().tofile(f)

with open(calculationsDir + "j_npv.bin", "wb+") as f:
    getNegativePredictiveValue_j(df).to_numpy().tofile(f)
    
with open(calculationsDir + "stat_parity.bin", "wb+") as f:
    get_statistical_parity(df).to_numpy().tofile(f)

with open(calculationsDir + "disp_impact.bin", "wb+") as f:
    get_disparate_impact(df).to_numpy().tofile(f)

with open(calculationsDir + "acc_equality_ratio.bin", "wb+") as f:
    get_acc_equality_ratio(df).to_numpy().tofile(f)

with open(calculationsDir + "acc_equality_diff.bin", "wb+") as f:
    get_acc_equality_diff(df).to_numpy().tofile(f)
    
# Free the memory
del df
gc.collect()
gc.get_stats()

[{'collections': 378, 'collected': 3676, 'uncollectable': 0},
 {'collections': 36, 'collected': 314, 'uncollectable': 0},
 {'collections': 4, 'collected': 37, 'uncollectable': 0}]

## Append calculations of the 2st half of the dataset

In [4]:
with open(datasetName, "rb") as f:
    df = pd.DataFrame(pickle.load(f), columns = dataCols)

halfIdx = int(df.shape[0] / 2)
df = df.iloc[halfIdx:]

In [5]:
with open(calculationsDir + "gr.bin", "ab+") as f:
    getGroupRatios(df).to_numpy().tofile(f)

with open(calculationsDir + "ir.bin", "ab+") as f:
    getImbalanceRatios(df).to_numpy().tofile(f)

with open(calculationsDir + "i_tpr.bin", "ab+") as f:
    getTPR_i(df).to_numpy().tofile(f)

with open(calculationsDir + "j_tpr.bin", "ab+") as f:
    getTPR_j(df).to_numpy().tofile(f)

with open(calculationsDir + "i_fpr.bin", "ab+") as f:
    getFPR_i(df).to_numpy().tofile(f)

with open(calculationsDir + "j_fpr.bin", "ab+") as f:
    getFPR_j(df).to_numpy().tofile(f)

with open(calculationsDir + "i_ppv.bin", "ab+") as f:
    getPositivePredictiveValue_i(df).to_numpy().tofile(f)

with open(calculationsDir + "j_ppv.bin", "ab+") as f:
    getPositivePredictiveValue_j(df).to_numpy().tofile(f)

with open(calculationsDir + "i_npv.bin", "ab+") as f:
    getNegativePredictiveValue_i(df).to_numpy().tofile(f)

with open(calculationsDir + "j_npv.bin", "ab+") as f:
    getNegativePredictiveValue_j(df).to_numpy().tofile(f)
    
with open(calculationsDir + "stat_parity.bin", "ab+") as f:
    get_statistical_parity(df).to_numpy().tofile(f)

with open(calculationsDir + "disp_impact.bin", "ab+") as f:
    get_disparate_impact(df).to_numpy().tofile(f)

with open(calculationsDir + "acc_equality_ratio.bin", "ab+") as f:
    get_acc_equality_ratio(df).to_numpy().tofile(f)

with open(calculationsDir + "acc_equality_diff.bin", "ab+") as f:
    get_acc_equality_diff(df).to_numpy().tofile(f)
    
del df
gc.collect()
gc.get_stats()

[{'collections': 376, 'collected': 3669, 'uncollectable': 0},
 {'collections': 36, 'collected': 300, 'uncollectable': 0},
 {'collections': 4, 'collected': 37, 'uncollectable': 0}]

In [6]:
!ls -lah calculations

total 58G
drwxr-xr-x 2 root root 4.0K Nov 27 19:47 .
drwxr-xr-x 1 root root 4.0K Nov 27 19:45 ..
-rw-r--r-- 1 root root 4.2G Nov 27 19:50 acc_equality_diff.bin
-rw-r--r-- 1 root root 4.2G Nov 27 19:50 acc_equality_ratio.bin
-rw-r--r-- 1 root root 4.2G Nov 27 19:50 disp_impact.bin
-rw-r--r-- 1 root root 4.2G Nov 27 19:48 gr.bin
-rw-r--r-- 1 root root 4.2G Nov 27 19:49 i_fpr.bin
-rw-r--r-- 1 root root 4.2G Nov 27 19:49 i_npv.bin
-rw-r--r-- 1 root root 4.2G Nov 27 19:49 i_ppv.bin
-rw-r--r-- 1 root root 4.2G Nov 27 19:48 ir.bin
-rw-r--r-- 1 root root 4.2G Nov 27 19:48 i_tpr.bin
-rw-r--r-- 1 root root 4.2G Nov 27 19:49 j_fpr.bin
-rw-r--r-- 1 root root 4.2G Nov 27 19:50 j_npv.bin
-rw-r--r-- 1 root root 4.2G Nov 27 19:49 j_ppv.bin
-rw-r--r-- 1 root root 4.2G Nov 27 19:49 j_tpr.bin
-rw-r--r-- 1 root root 4.2G Nov 27 19:50 stat_parity.bin


The files are written as `float64`, while the dataset has data-type `int8`, thus the size of each file is the same as the size of the dataset because each of them contains one 8 times heavier column.

# Get additional calculations
These calculations will be based on the previous ones.

In [ ]:
#equal opportunity ratio of true positive rates
def get_equal_opp_ratio(j_tpr, i_tpr):
    return j_tpr / i_tpr

#equal opportunity difference of true positive rates
def get_equal_opp_diff(j_tpr, i_tpr):
    return j_tpr - i_tpr

#predictive equality ratio
def get_pred_equality_ratio(j_fpr, i_fpr):
    return j_fpr / i_fpr

#predictive equality difference
def get_pred_equality_diff(j_fpr, i_fpr):
    return j_fpr - i_fpr

#positive predictive parity ratio
def get_pred_parity_ratio(j_ppv, i_ppv):
    return j_ppv / i_ppv

#positive predictive parity difference
def get_pos_pred_parity_diff(j_ppv, i_ppv):
    return j_ppv - i_ppv

#negative predictive parity difference
def get_neg_pred_parity_ratio(j_npv, i_npv):
    return j_npv / i_npv

#negative predictive parity difference
def get_neg_pred_parity_diff(j_npv, i_npv):
    return j_npv - i_npv

## Write 1st part
Here the story is even worse as we need to open 2 files of the same size, so we will do it the same way: in 2 stages.

In [ ]:
with open(calculationsDir + "i_tpr.bin", "rb") as f:
    i_tpr = pd.DataFrame(np.fromfile(f), columns = ["i_tpr"])
    halfIdx = int(i_tpr.shape[0] / 2)
    i_tpr = i_tpr.iloc[:halfIdx]

with open(calculationsDir + "j_tpr.bin", "rb") as f:
    j_tpr = pd.DataFrame(np.fromfile(f), columns = ["j_tpr"])
    halfIdx = int(j_tpr.shape[0] / 2)
    j_tpr = j_tpr.iloc[:halfIdx]
    
with open(calculationsDir + "equal_opp_ratio.bin", "wb+") as f:
    get_equal_opp_ratio(j_tpr['j_tpr'], i_tpr['i_tpr']).to_numpy().tofile(f)
    
with open(calculationsDir + "equal_opp_diff.bin", "wb+") as f:
    get_equal_opp_diff(j_tpr['j_tpr'], i_tpr['i_tpr']).to_numpy().tofile(f)

del j_tpr
del i_tpr
gc.collect()

In [ ]:
with open(calculationsDir + "i_fpr.bin", "rb") as f:
    i_fpr = pd.DataFrame(np.fromfile(f), columns = ["i_fpr"])
    halfIdx = int(i_fpr.shape[0] / 2)
    i_fpr = i_fpr.iloc[:halfIdx]

with open(calculationsDir + "j_fpr.bin", "rb") as f:
    j_fpr = pd.DataFrame(np.fromfile(f), columns = ["j_fpr"])
    halfIdx = int(j_fpr.shape[0] / 2)
    j_fpr = j_fpr.iloc[:halfIdx]

with open(calculationsDir + "pred_equality_ratio.bin", "wb+") as f:
    get_pred_equality_ratio(j_fpr['j_fpr'], i_fpr['i_fpr']).to_numpy().tofile(f)
    
with open(calculationsDir + "pred_equality_diff.bin", "wb+") as f:
    get_pred_equality_diff(j_fpr['j_fpr'], i_fpr['i_fpr']).to_numpy().tofile(f)
    
del j_fpr
del i_fpr
gc.collect()

In [ ]:
with open(calculationsDir + "i_ppv.bin", "rb") as f:
    i_ppv = pd.DataFrame(np.fromfile(f), columns = ["i_ppv"])
    halfIdx = int(i_ppv.shape[0] / 2)
    i_ppv = i_ppv.iloc[:halfIdx]
    
with open(calculationsDir + "j_ppv.bin", "rb") as f:
    j_ppv = pd.DataFrame(np.fromfile(f), columns = ["j_ppv"])
    halfIdx = int(j_ppv.shape[0] / 2)
    j_ppv = j_ppv.iloc[:halfIdx]

with open(calculationsDir + "pred_parity_ratio.bin", "wb+") as f:
    get_pred_parity_ratio(j_ppv['j_ppv'], i_ppv['i_ppv']).to_numpy().tofile(f)

with open(calculationsDir + "pos_pred_parity_diff.bin", "wb+") as f:
    get_pos_pred_parity_diff(j_ppv['j_ppv'], i_ppv['i_ppv']).to_numpy().tofile(f)

del j_ppv
del i_ppv
gc.collect()

In [ ]:
with open(calculationsDir + "i_npv.bin", "rb") as f:
    i_npv = pd.DataFrame(np.fromfile(f), columns = ["i_npv"])
    halfIdx = int(i_npv.shape[0] / 2)
    i_npv = i_npv.iloc[:halfIdx]

with open(calculationsDir + "j_npv.bin", "rb") as f:
    j_npv = pd.DataFrame(np.fromfile(f), columns = ["j_npv"])
    halfIdx = int(j_npv.shape[0] / 2)
    j_npv = j_npv.iloc[:halfIdx]

with open(calculationsDir + "neg_pred_parity_ratio.bin", "wb+") as f:
    get_neg_pred_parity_ratio(j_npv['j_npv'], i_npv['i_npv']).to_numpy().tofile(f)

with open(calculationsDir + "neg_pred_parity_diff.bin", "wb+") as f:
    get_neg_pred_parity_diff(j_npv['j_npv'], i_npv['i_npv']).to_numpy().tofile(f)

del j_npv
del i_npv
gc.collect()

## Append 2nd part

In [ ]:
with open(calculationsDir + "i_tpr.bin", "rb") as f:
    i_tpr = pd.DataFrame(np.fromfile(f), columns = ["i_tpr"])
    halfIdx = int(i_tpr.shape[0] / 2)
    i_tpr = i_tpr.iloc[halfIdx:]

with open(calculationsDir + "j_tpr.bin", "rb") as f:
    j_tpr = pd.DataFrame(np.fromfile(f), columns = ["j_tpr"])
    halfIdx = int(j_tpr.shape[0] / 2)
    j_tpr = j_tpr.iloc[halfIdx:]
    
with open(calculationsDir + "equal_opp_ratio.bin", "ab+") as f:
    get_equal_opp_ratio(j_tpr['j_tpr'], i_tpr['i_tpr']).to_numpy().tofile(f)
    
with open(calculationsDir + "equal_opp_diff.bin", "ab+") as f:
    get_equal_opp_diff(j_tpr['j_tpr'], i_tpr['i_tpr']).to_numpy().tofile(f)

del j_tpr
del i_tpr
gc.collect()

In [ ]:
with open(calculationsDir + "i_fpr.bin", "rb") as f:
    i_fpr = pd.DataFrame(np.fromfile(f), columns = ["i_fpr"])
    halfIdx = int(i_fpr.shape[0] / 2)
    i_fpr = i_fpr.iloc[halfIdx:]

with open(calculationsDir + "j_fpr.bin", "rb") as f:
    j_fpr = pd.DataFrame(np.fromfile(f), columns = ["j_fpr"])
    halfIdx = int(j_fpr.shape[0] / 2)
    j_fpr = j_fpr.iloc[halfIdx:]

with open(calculationsDir + "pred_equality_ratio.bin", "ab+") as f:
    get_pred_equality_ratio(j_fpr['j_fpr'], i_fpr['i_fpr']).to_numpy().tofile(f)
    
with open(calculationsDir + "pred_equality_diff.bin", "ab+") as f:
    get_pred_equality_diff(j_fpr['j_fpr'], i_fpr['i_fpr']).to_numpy().tofile(f)
    
del j_fpr
del i_fpr
gc.collect()

In [ ]:
with open(calculationsDir + "i_ppv.bin", "rb") as f:
    i_ppv = pd.DataFrame(np.fromfile(f), columns = ["i_ppv"])
    halfIdx = int(i_ppv.shape[0] / 2)
    i_ppv = i_ppv.iloc[halfIdx:]
    
with open(calculationsDir + "j_ppv.bin", "rb") as f:
    j_ppv = pd.DataFrame(np.fromfile(f), columns = ["j_ppv"])
    halfIdx = int(j_ppv.shape[0] / 2)
    j_ppv = j_ppv.iloc[halfIdx:]

with open(calculationsDir + "pred_parity_ratio.bin", "ab+") as f:
    get_pred_parity_ratio(j_ppv['j_ppv'], i_ppv['i_ppv']).to_numpy().tofile(f)

with open(calculationsDir + "pos_pred_parity_diff.bin", "ab+") as f:
    get_pos_pred_parity_diff(j_ppv['j_ppv'], i_ppv['i_ppv']).to_numpy().tofile(f)

del j_ppv
del i_ppv
gc.collect()

In [ ]:
with open(calculationsDir + "i_npv.bin", "rb") as f:
    i_npv = pd.DataFrame(np.fromfile(f), columns = ["i_npv"])
    halfIdx = int(i_npv.shape[0] / 2)
    i_npv = i_npv.iloc[:halfIdx]

with open(calculationsDir + "j_npv.bin", "rb") as f:
    j_npv = pd.DataFrame(np.fromfile(f), columns = ["j_npv"])
    halfIdx = int(j_npv.shape[0] / 2)
    j_npv = j_npv.iloc[:halfIdx]

with open(calculationsDir + "neg_pred_parity_ratio.bin", "ab+") as f:
    get_neg_pred_parity_ratio(j_npv['j_npv'], i_npv['i_npv']).to_numpy().tofile(f)

with open(calculationsDir + "neg_pred_parity_diff.bin", "ab+") as f:
    get_neg_pred_parity_diff(j_npv['j_npv'], i_npv['i_npv']).to_numpy().tofile(f)

del j_npv
del i_npv
gc.collect()

## Calculations for sonya plot

In [4]:
# diff_metrics = [f.split('\\')[1] for f in glob.glob(calculationsDir+'*diff.bin')]
diff_metrics = {'acc_equality_diff.bin': 'Accuracy equality difference', 
               'equal_opp_diff.bin': 'Equal opportunity difference', 
               'neg_pred_parity_diff.bin': 'Negative predictive parity difference', 
               'pos_pred_parity_diff.bin': 'Positive predictive parity difference', 
               'pred_equality_diff.bin': 'Predictive equality difference', 
               }

In [5]:
def calculate_sonya(df, base_metric):   
    diff_probs = {}
    compute_diff_prob = lambda df: np.sum(df['diff'] == 0) / len(df)
    
    for metricFName in diff_metrics:
        with open(metricFName, "rb") as f:
            diff_metric = pd.DataFrame(np.fromfile(f).astype(np.float16), columns = ["diff"])
        df = pd.concat([df, diff_metric], axis = 1)

        diff = df.groupby(base_metric).apply(compute_diff_prob)
        diff_probs[diff_metrics[metricFName]] = diff

        df.drop('diff', axis=1, inplace=True)
        
    sonya = pd.DataFrame(diff_probs)
    sonya *= 100
    sonya.reset_index(inplace=True)
    sonya.to_csv(calculationsDir + "sonya_" + base_metric + ".csv", index=False)

In [4]:
with open("ir.bin", "rb") as f:
    df = pd.DataFrame(np.fromfile(f).astype(np.float16), columns = ["ir"])
calculate_sonya(df, 'ir')

In [8]:
with open("gr.bin", "rb") as f:
    df = pd.DataFrame(np.fromfile(f).astype(np.float16), columns = ["gr"])
calculate_sonya(df, 'gr')

In [16]:
# stat_parity = ((df.j_tp + df.j_fp)/(df.j_tp + df.j_fp + df.j_tn + df.j_fn)) - ((df.i_tp + df.i_fp)/(df.i_tp + df.i_fp + df.i_tn + df.i_fn))
# stereotypical bias = abs(Mpos/totalM - Fpos/totalF)
# abs(stat_parity) = stereotypical bias 
with open("stat_parity.bin", "rb") as f:
    df = pd.DataFrame(np.fromfile(f).astype(np.float16), columns = ["st_bs"])
calculate_sonya(abs(df), 'st_bs')